In [2]:
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive/NLP\ project/models/
%cd /content/drive/MyDrive/SMU_MITB_NLP/NLP-Lyric-Generator/src/bin/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_NLP/NLP-Lyric-Generator/src/bin


In [40]:
#import packages
import pandas as pd
import numpy as np
import os
import sys 
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Data Preprocessing

In [7]:
# from google.colab import files
# src = list(files.upload().values())[0]
# open('utilities.py','wb').write(src)
# import utilities as U

### Custom Imports
import os
import sys
sys.path.append('../')
import lib.utilities as U
from lib.sentiment import Sentiment

In [9]:
#load train and validation corpus
train_corpus, val_corpus, train_file, val_file = U.split_corpus(path = '/content/drive/MyDrive/SMU_MITB_NLP/NLP-Lyric-Generator/data/', val_split = 0.2, end_song_token = '\n\n<EOS>\n\n', random_seed = 2022)

In [10]:
train_corpus[:100]

"<VERSE>\nThis is a place\nYou'll be amazed\nWhere children play\nWhere best friends stay\nHave different "

Data Preprocessing for Train Corpus

In [11]:
train_components = train_corpus.split('\n\n')

In [12]:
print(len(train_components))

272


In [13]:
train_corpus_list = [string.split('\n') for string in train_components]
for song in train_corpus_list:
  song.pop(0)

In [14]:
train_corpus_list[:5]

[['This is a place',
  "You'll be amazed",
  'Where children play',
  'Where best friends stay',
  'Have different tastes',
  'Live different ways',
  'The smile on your face',
  'Can bring us closer every day'],
 ['This is a place',
  'Our common space',
  'We share our dreams',
  'One goal we chase',
  'Come join our pace',
  "We'll win the race",
  "When we're together",
  'We can make miracles come our way'],
 ["This is where we'll see",
  'Familiar faces from our neighbourhood',
  "This is where we'll share",
  'The stories dating back to our childhood',
  "It's here where I'll find",
  'Your hand to lift me up',
  'When I fall',
  "It's here where I am sure",
  "I'm safe",
  "I won't be lost"],
 ["This is where we'll stand as one",
  'To build a home for you and me',
  "This is where we'll join our hands",
  'To grow the fruits of harmony',
  'When you and I',
  'Put our hearts together',
  "We'll stand the test",
  'For this is where we belong',
  "We'll give our best"],
 ['This

In [15]:
df_train = pd.DataFrame({'song': train_corpus_list})

df_train['song'] = df_train['song'].str.join(', ')
pd.options.display.max_colwidth = 500
df_train=df_train.iloc[1:] 
df_train.head()

,song
1,"This is a place, Our common space, We share our dreams, One goal we chase, Come join our pace, We'll win the race, When we're together, We can make miracles come our way"
2,"This is where we'll see, Familiar faces from our neighbourhood, This is where we'll share, The stories dating back to our childhood, It's here where I'll find, Your hand to lift me up, When I fall, It's here where I am sure, I'm safe, I won't be lost"
3,"This is where we'll stand as one, To build a home for you and me, This is where we'll join our hands, To grow the fruits of harmony, When you and I, Put our hearts together, We'll stand the test, For this is where we belong, We'll give our best"
4,"This is the place, Our common space, We share our dreams, One goal we chase, Come join our pace, We'll win the race, Here's where I belong, Where I know that love is there for me"
5,"This is where we'll see, Familiar faces from our neighbourhood, This is where we'll share, The stories dating back to our childhood, It's here where I'll find, Your hand to lift me up, When I fall, It's here where I am sure, I'm safe, I won't be lost"


In [16]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

df_train['song'] = df_train.apply(lambda row : U.remove_punct(row['song']), axis=1)
df_train['song'] = df_train.apply(lambda row : U.remove_url(row['song']), axis=1)
df_train['song'] = df_train.apply(lambda row : U.remove_emoji(row['song']), axis=1)
df_train['song'] = df_train.apply(lambda row : U.decontraction(row['song']), axis=1)
df_train['song']=  df_train.apply(lambda row : row['song'].lower(), axis=1)
df_train.head()

,song
1,this is a place our common space we share our dreams one goal we chase come join our pace well win the race when were together we can make miracles come our way
2,this is where well see familiar faces from our neighbourhood this is where well share the stories dating back to our childhood its here where ill find your hand to lift me up when i fall its here where i am sure im safe i wont be lost
3,this is where well stand as one to build a home for you and me this is where well join our hands to grow the fruits of harmony when you and i put our hearts together well stand the test for this is where we belong well give our best
4,this is the place our common space we share our dreams one goal we chase come join our pace well win the race heres where i belong where i know that love is there for me
5,this is where well see familiar faces from our neighbourhood this is where well share the stories dating back to our childhood its here where ill find your hand to lift me up when i fall its here where i am sure im safe i wont be lost


In [17]:
train_songs = df_train['song'].tolist()

In [18]:
# Save train song set in .txt file
with open('train_songs.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in train_songs:
        filehandle.write('%s\n' % listitem)

In [19]:
textFileName = 'train_songs.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()

Data Preprocessing for Validation Corpus

In [20]:
val_components = val_corpus.split('\n\n')

In [21]:
print(len(val_components))

67


In [22]:
val_corpus_list = [string.split('\n') for string in val_components]
for song in val_corpus_list:
  song.pop(0)

In [24]:
val_corpus_list[:5]

[['There was a time when people said',
  "that Singapore won't make it ",
  'but we did',
  'There was a time when troubles seemed ',
  'too much for us to take',
  'but we did',
  'We built a nation, ',
  'strong and free',
  'Reaching out together',
  'For peace and harmony'],
 ['This is my country, ',
  'this is my flag',
  'This is my future, ',
  'this is my life',
  'This is my family, ',
  'these are my friends',
  'We are Singapore, ',
  'Singaporeans',
  'Singapore our homeland, ',
  "it's here that we belong",
  'All of us united, ',
  'one people marching on',
  "We've come so far together, ",
  'our common destiny',
  'Singapore forever, ',
  'a nation strong and free',
  'This is my country, ',
  'this is my flag',
  'This is my future, ',
  'this is my life',
  'This is my family, ',
  'these are my friends',
  'We are Singapore, ',
  'Singaporeans'],
 ['We the citizens of Singapore',
  'Pledge ourselves as one united people',
  'Regardless of race language or religion',


In [25]:
df_val = pd.DataFrame({'song': val_corpus_list})

df_val['song'] = df_val['song'].str.join(', ')
df_val=df_val.iloc[1:] 
df_val.head()

,song
1,"This is my country, , this is my flag, This is my future, , this is my life, This is my family, , these are my friends, We are Singapore, , Singaporeans, Singapore our homeland, , it's here that we belong, All of us united, , one people marching on, We've come so far together, , our common destiny, Singapore forever, , a nation strong and free, This is my country, , this is my flag, This is my future, , this is my life, This is my family, , these are my friends, We are Singapore, , Singaporeans"
2,"We the citizens of Singapore, Pledge ourselves as one united people, Regardless of race language or religion, To build a democratic society, Based on justice and equality, So as to achieve happiness, Prosperity and progress for our nation"
3,"We are Singapore, , we are Singapore, We will stand together, , hear the lion roar, We are Singapore, , we are Singapore, We're a nation strong and free , forevermore, This is my country, , this is my flag, This is my future, , this is my life, This is my family, , these are my friends, We are Singapore, , Singaporeans, We are Singapore, , we are Singapore, We will stand together, , hear the lion roar, We are Singapore, , we are Singapore, We're a nation strong and free , forevermore, We are..."
4,<EOS>
5,"I share this thought with everyone, Escape your ordinary world, Reach out your hand to others and, Your dreams will live, They'll be unfurled"


In [26]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

df_val['song'] = df_val.apply(lambda row : U.remove_punct(row['song']), axis=1)
df_val['song'] = df_val.apply(lambda row : U.remove_url(row['song']), axis=1)
df_val['song'] = df_val.apply(lambda row : U.remove_emoji(row['song']), axis=1)
df_val['song'] = df_val.apply(lambda row : U.decontraction(row['song']), axis=1)
df_val['song']=  df_val.apply(lambda row : row['song'].lower(), axis=1)
df_val.head()

,song
1,this is my country this is my flag this is my future this is my life this is my family these are my friends we are singapore singaporeans singapore our homeland its here that we belong all of us united one people marching on weve come so far together our common destiny singapore forever a nation strong and free this is my country this is my flag this is my future this is my life this is my family these are my friends we are singapore singaporeans
2,we the citizens of singapore pledge ourselves as one united people regardless of race language or religion to build a democratic society based on justice and equality so as to achieve happiness prosperity and progress for our nation
3,we are singapore we are singapore we will stand together hear the lion roar we are singapore we are singapore were a nation strong and free forevermore this is my country this is my flag this is my future this is my life this is my family these are my friends we are singapore singaporeans we are singapore we are singapore we will stand together hear the lion roar we are singapore we are singapore were a nation strong and free forevermore we are singapore singaporeans we are singapore singapo...
4,eos
5,i share this thought with everyone escape your ordinary world reach out your hand to others and your dreams will live theyll be unfurled


In [27]:
val_songs = df_val['song'].tolist()

In [28]:
# Save train song set in .txt file
with open('val_songs.txt', 'w',encoding="utf-8") as valfilehandle:  
    for listitem in val_songs:
        valfilehandle.write('%s\n' % listitem)

In [29]:
textFileName = 'val_songs.txt'
val_raw_text = open(textFileName, encoding = 'UTF-8').read()

In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
import keras.utils as ku 

val_tokenizer = Tokenizer()

val_corpus = raw_text.lower().split("\n")
val_tokenizer.fit_on_texts(val_corpus)
val_total_words = len(val_tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in val_corpus:
    token_list = val_tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
val_max_sequence_len = max([len(x) for x in input_sequences])
val_input_sequences = np.array(pad_sequences(input_sequences, maxlen=val_max_sequence_len, padding='pre'))
 
# create predictors and label
val_predictors, val_label = val_input_sequences[:,:-1],val_input_sequences[:,-1]
 
val_label = ku.np_utils.to_categorical(val_label, num_classes=val_total_words)

Word-Level Lyrics Generation

In [32]:
"""
Generate song.
Reference: https://predictivehacks.com/word-based-text-generation/
"""


tokenizer = Tokenizer()

train_corpus = raw_text.lower().split("\n")
tokenizer.fit_on_texts(train_corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in train_corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.np_utils.to_categorical(label, num_classes=total_words)

#set SEED 
tf.random.set_seed(42)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 96, 100)           97100     
                                                                 
 dropout (Dropout)           (None, 96, 100)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               29312     
                                                                 
 dense (Dense)               (None, 485)               62565     
                                                                 
 dense_1 (Dense)             (None, 971)               471906    
                                                                 
Total params: 660,883
Trainable params: 660,883
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
#Implement early stopping
early_stop= tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [34]:
history = model.fit(predictors, label, epochs=30, verbose=1,callbacks = [early_stop])

Epoch 1/30
257/257 [==============================] - 15s 54ms/step - loss: 6.2007 - accuracy: 0.0397
Epoch 2/30
257/257 [==============================] - 13s 51ms/step - loss: 5.3353 - accuracy: 0.0629
Epoch 3/30
257/257 [==============================] - 13s 51ms/step - loss: 4.9964 - accuracy: 0.0816
Epoch 4/30
257/257 [==============================] - 13s 50ms/step - loss: 4.7248 - accuracy: 0.1027
Epoch 5/30
257/257 [==============================] - 13s 52ms/step - loss: 4.4736 - accuracy: 0.1331
Epoch 6/30
257/257 [==============================] - 13s 51ms/step - loss: 4.2367 - accuracy: 0.1639
Epoch 7/30
257/257 [==============================] - 13s 50ms/step - loss: 3.9958 - accuracy: 0.1992
Epoch 8/30
257/257 [==============================] - 13s 50ms/step - loss: 3.7737 - accuracy: 0.2430
Epoch 9/30
257/257 [==============================] - 13s 52ms/step - loss: 3.5586 - accuracy: 0.2836
Epoch 10/30
257/257 [==============================] - 14s 55ms/step - loss: 3.327

In [35]:
#Compute perplexity
val_loss, val_accuracy = model.evaluate(val_predictors, val_label)
print("Validation loss: {:.4f}".format(val_loss))
print("Validation accuracy: {:.4f}".format(val_accuracy))

257/257 [==============================] - 4s 14ms/step - loss: 0.9197 - accuracy: 0.8943
Validation loss: 0.9197
Validation accuracy: 0.8943


In [36]:
import math
perplexity = math.exp(val_loss)
print("Perplexity Score: ", perplexity)

Perplexity Score:  2.5085753121599073


Generated lyrics and evaluate sentiment of generated lyrics

In [4]:
### Custom Imports
# import os
# import sys
# sys.path.append('../')
# import lib.utilities as utils
# from lib.sentiment import Sentiment

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Input: Whenever I think back

In [37]:
'''
Neutral input seeds:
Whenever I think back 
And so this I know 
'''

seed_text = "whenever i think back"
next_words = 100

def generateLyrics(seed_text, model):
  next_words = 100   
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted=np.argmax(model.predict(token_list, verbose=0) ,axis=1)

      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              # break
      seed_text += " " + output_word
  return seed_text

lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['whenever i think back and i look around us and i am sure im safe i wont be lost from our destiny that we shine on me in us find right on our faces heart to safe and want the sunrise to a sky and a beautiful so set that we are weve always darkest are we have kept kept around us by you see yeah cant you see ooh you join my friends and enemies i guess beginning for you are this land where we belong well make grow were other some words last light is in ready us and i go a']

In [38]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("whenever i think back.txt", header=False, index=False, encoding="utf-8")

from google.colab import files
files.download("whenever i think back.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['whenever', 'think', 'back', 'look', 'around', 'us', 'sure', 'im', 'safe', 'wont', 'lost', 'destiny', 'shine', 'us', 'find', 'right', 'faces', 'heart', 'safe', 'want', 'sunrise', 'sky', 'beautiful', 'set', 'weve', 'always', 'darkest', 'kept', 'kept', 'around', 'us', 'see', 'yeah', 'cant', 'see', 'ooh', 'join', 'friends', 'enemies', 'guess', 'beginning', 'land', 'belong', 'well', 'make', 'grow', 'words', 'last', 'light', 'ready', 'us', 'go']


In [45]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.114, 'neu': 0.538, 'pos': 0.347, 'compound': 0.9309}}


In [46]:
from gensim import downloader
glove_vectors = downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [55]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [56]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.24324052, 'love': 0.85279125}}


Input: and so this i know

In [57]:
seed_text = "and so this i know"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['and so this i know its singapore sing a song for singapore with every generation theres more to be grateful for so come and sing sing a song for singapore youre my brother youre my sister im thankful for my singapore my singapore singapore i love singapore i love singapore i love singapore singapore i love you i love you and so much more together hand in hand in joy and harmony we will strive each day be the best that we can be growing stronger every day well make it you and me well work together hand in hand in joy and harmony we']

In [50]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("and so this I know.txt", header=False, index=False, encoding="utf-8")
files.download("and so this I know.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['know', 'singapore', 'sing', 'song', 'singapore', 'every', 'generation', 'theres', 'grateful', 'come', 'sing', 'sing', 'song', 'singapore', 'youre', 'brother', 'youre', 'sister', 'im', 'thankful', 'singapore', 'singapore', 'singapore', 'love', 'singapore', 'love', 'singapore', 'love', 'singapore', 'singapore', 'love', 'love', 'much', 'together', 'hand', 'hand', 'joy', 'harmony', 'strive', 'day', 'best', 'growing', 'stronger', 'every', 'day', 'well', 'make', 'well', 'work', 'together', 'hand', 'hand', 'joy', 'harmony']


In [59]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'compound': 0.9965}}


In [60]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [61]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.24324052, 'love': 0.85279125}}


Input: I am tired of being what you want me to be

In [62]:
'''
Negative input seeds:
I am tired of being what you want me to be 
Feeling so faithless, lost under the surface 
'''

seed_text = "i am tired of being what you want me to be"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['i am tired of being what you want me to be so ways and a faraway isle turning darkness into light rain the sour light rain still wonderful youre i fall its has singing things we can achieve together we can be a part for you and me to show our part for singapore in me i love you know made our childhood lies my homeland i will share a part for here and all a goal to reach out for our best will be happiness of who and you just me nothing like i think of the best know never built do do you and i see a place to']

In [63]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("i am tired of being what you want me to be.txt", header=False, index=False, encoding="utf-8")
files.download("i am tired of being what you want me to be.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['tired', 'want', 'ways', 'faraway', 'isle', 'turning', 'darkness', 'light', 'rain', 'sour', 'light', 'rain', 'still', 'wonderful', 'youre', 'fall', 'singing', 'things', 'achieve', 'together', 'part', 'show', 'part', 'singapore', 'love', 'know', 'made', 'childhood', 'lies', 'homeland', 'share', 'part', 'goal', 'reach', 'best', 'happiness', 'nothing', 'like', 'think', 'best', 'know', 'never', 'built', 'see', 'place']


In [65]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.146, 'neu': 0.49, 'pos': 0.364, 'compound': 0.9402}}


In [66]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [67]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.2918983, 'love': 0.9043216}}


Input: feeling so faithless lost under the surface

In [70]:
seed_text = "feeling so faithless lost under the surface"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['feeling so faithless lost under the surface years they been longing to see this land that well share a special dream together we feel the singapore heartbeat we can reach for our dreams come there moments of us and hold a nation much for the beautiful feeling rain the beautiful signs to do it believe what do you see what do you see it like us come on them on a dream together we can make the change and an unforgettable sight to see the sunrise on a faraway isle turning darkness into light rain the sour wall in my hour experienced sweet and sour but thats']

In [71]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("feeling so faithless lost under the surface.txt", header=False, index=False, encoding="utf-8")
files.download("feeling so faithless lost under the surface.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['feeling', 'faithless', 'lost', 'surface', 'years', 'longing', 'see', 'land', 'well', 'share', 'special', 'dream', 'together', 'feel', 'singapore', 'heartbeat', 'reach', 'dreams', 'come', 'moments', 'us', 'hold', 'nation', 'much', 'beautiful', 'feeling', 'rain', 'beautiful', 'signs', 'believe', 'see', 'see', 'like', 'us', 'come', 'dream', 'together', 'make', 'change', 'unforgettable', 'sight', 'see', 'sunrise', 'faraway', 'isle', 'turning', 'darkness', 'light', 'rain', 'sour', 'wall', 'hour', 'experienced', 'sweet', 'sour', 'thats']


In [73]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.095, 'neu': 0.503, 'pos': 0.401, 'compound': 0.967}}


In [74]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [75]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.27498844, 'love': 0.88918775}}


Input: relight our fire we will find our way

In [76]:
'''
Positive input seeds:
Relight our fire, we will find our way 
We will rise stronger together  
'''
seed_text = "relight our fire we will find our way"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['relight our fire we will find our way on the great colours lighting up the sky we are the ones world where i am i will can achieve together we can be a part for me and all to keep my dreams and many moments of who and mirth where ill find it we go through a great wall in our way on a thousand colours lighting up the sky we are the ones world where i am i will can achieve together we can be a part for me and all to keep my dreams and many moments of who and mirth where ill find it we']

In [77]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("relight our fire we will find our way.txt", header=False, index=False, encoding="utf-8")
files.download("relight our fire we will find our way.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['relight', 'fire', 'find', 'way', 'great', 'colours', 'lighting', 'sky', 'ones', 'world', 'achieve', 'together', 'part', 'keep', 'dreams', 'many', 'moments', 'mirth', 'ill', 'find', 'go', 'great', 'wall', 'way', 'thousand', 'colours', 'lighting', 'sky', 'ones', 'world', 'achieve', 'together', 'part', 'keep', 'dreams', 'many', 'moments', 'mirth', 'ill', 'find']


In [79]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.134, 'neu': 0.518, 'pos': 0.348, 'compound': 0.93}}


In [80]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [81]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.3006478, 'love': 0.87343216}}


Input: we will rise stronger together

In [82]:
seed_text = "we will rise stronger together"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['we will rise stronger together you feel me to made you are happiness for in i take it treasure right on me well show our part together well make our faces stand and goals in a world our hearts will destiny we all achieve what my singapore with well grow we are our people lets into the ones we have a road accustomed we feel that you brave the whole world where i am sure im heard a destiny and tomorrow become like to never be kept look around us like and say it but dont with all too bold its the sea when we']

In [83]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("we will rise stronger together.txt", header=False, index=False, encoding="utf-8")
files.download("we will rise stronger together.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
# create Sentiment object
sentiment = Sentiment()

train_songs_str = ' '.join([str(elem) for elem in train_songs])
lyrics_str = ' '.join([str(elem) for elem in lyrics_list])

# clean and tokenize text
sentiment.clean_text(
    original_text = train_songs_str, 
    generated_text = lyrics_str, 
    remove_stopwords=True, 
    stop_words = set(stopwords.words("english"))
    )

print(sentiment.original_tokens[:10])
print(sentiment.generated_tokens)

['place', 'common', 'space', 'share', 'dreams', 'one', 'goal', 'chase', 'come', 'join']
['rise', 'stronger', 'together', 'feel', 'made', 'happiness', 'take', 'treasure', 'right', 'well', 'show', 'part', 'together', 'well', 'make', 'faces', 'stand', 'goals', 'world', 'hearts', 'destiny', 'achieve', 'singapore', 'well', 'grow', 'people', 'lets', 'ones', 'road', 'accustomed', 'feel', 'brave', 'whole', 'world', 'sure', 'im', 'heard', 'destiny', 'tomorrow', 'become', 'like', 'never', 'kept', 'look', 'around', 'us', 'like', 'say', 'dont', 'bold', 'sea']


In [85]:
# this step scores the vader sentiment of the original and generated text 
sentiment.score_vader_sentiment()  
print(sentiment.vader_sentiment_scores)

{'original': {'neg': 0.028, 'neu': 0.615, 'pos': 0.357, 'compound': 1.0}, 'generated': {'neg': 0.032, 'neu': 0.592, 'pos': 0.376, 'compound': 0.9648}}


In [86]:
# this step extracts the top n similar word vectors to each theme
# and get their average word vector
sentiment.get_theme_vector(
    sentiment_themes=['patriot', 'love'], 
    embedding=glove_vectors, 
    topn=10
)
print(sentiment.sentiment_themes)
print(sentiment.all_theme_vectors)

['patriot', 'love']
{'patriot': array([-0.0932492 ,  0.2737941 ,  0.46496803, -0.1104055 , -0.4727427 ,
       -0.3695609 ,  0.59728706, -0.56995434, -0.49318305,  1.152441  ,
        0.20548022, -0.665005  , -0.35317498, -0.05724651,  0.29801202,
       -0.1887181 ,  0.46808147,  0.29274267, -0.3542009 , -0.390468  ,
        0.41724506, -0.3645263 , -0.02154259,  0.15602511,  0.3117882 ,
        0.2808048 , -0.40197968,  0.19433399, -0.207225  ,  0.41112152,
       -0.40770826,  0.41357788, -0.06724178,  0.2552355 ,  0.6959089 ,
       -0.0333486 ,  0.220113  , -0.5254942 ,  0.0344236 ,  0.4185118 ,
       -0.3748475 ,  0.12050854,  0.2848566 ,  0.11164705,  0.83036804,
       -0.4462192 ,  0.3798733 , -0.0214241 , -0.4064907 , -0.348634  ,
        0.475555  ,  0.6571389 ,  0.1032334 ,  0.5260991 ,  0.343434  ,
       -0.392311  ,  0.63653004,  0.13372497,  1.32073   ,  0.2860896 ,
        0.255205  ,  0.743705  ,  0.62672967,  0.29697728, -0.22062948,
        0.33026758,  0.0073686 ,

In [87]:
# this step scores the sentiment of the original and generated text 
# by comparing them against the theme word vectors using cos sim 
sentiment.score_word_vector_sentiment()  
print(sentiment.word_vector_sentiment_scores)

{'original': {'patriot': 0.2953495, 'love': 0.9091801}, 'generated': {'patriot': 0.32240644, 'love': 0.88512415}}
